In [ ]:
import pathlib

In [ ]:
import joblib
import kaggle

In [ ]:
root_dir = "../.."

In [ ]:
root_dir_path = pathlib.Path(root_dir)
models_dir_path = root_dir_path / "models"
model_path = models_dir_path / "reg.joblib"
submission_path = models_dir_path / "submission.csv.gz"

In [ ]:
model = joblib.load(model_path)

In [ ]:
kaggle.api.competition_submit(
    submission_path,
    message=str(model.best_score),
    competition="m5-forecasting-accuracy",
)